In [ ]:
import cv2
import dlib
import numpy as np
from imutils import face_utils

In [ ]:
face_detector = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# Eye landmarks indices
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']

# Function: midpoint of two points
def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

In [ ]:
def eye_aspect_ratio(eye):
    # Compute distances
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
def get_gaze_ratio(eye_points, facial_landmarks, frame_gray):
    # Eye region coordinates
    eye_region = np.array([(facial_landmarks.part(point).x, facial_landmarks.part(point).y) 
                            for point in eye_points], np.int32)
    # Create mask
    height, width = frame_gray.shape
    mask = np.zeros((height, width), np.uint8)
    cv2.polylines(mask, [eye_region], True, 255, 2)
    cv2.fillPoly(mask, [eye_region], 255)
    eye = cv2.bitwise_and(frame_gray, frame_gray, mask=mask)

    # Bounding rectangle
    min_x = np.min(eye_region[:, 0])
    max_x = np.max(eye_region[:, 0])
    min_y = np.min(eye_region[:, 1])
    max_y = np.max(eye_region[:, 1])
    gray_eye = eye[min_y: max_y, min_x: max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)

    # Split into left and right halves
    h, w = threshold_eye.shape
    left_side = threshold_eye[0:h, 0:int(w/2)]
    right_side = threshold_eye[0:h, int(w/2):w]

    # Count white pixels
    left_white = cv2.countNonZero(left_side)
    right_white = cv2.countNonZero(right_side)

    if right_white == 0:
        gaze_ratio = 1
    else:
        gaze_ratio = left_white / right_white
    return gaze_ratio

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_detector(gray)
    for face in faces:
        landmarks = landmark_predictor(gray, face)
        # Detect and draw face rectangle
        x1, y1 = face.left(), face.top()
        x2, y2 = face.right(), face.bottom()
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)

        # Gaze for each eye
        left_gaze = get_gaze_ratio(range(lStart, lEnd), landmarks, gray)
        right_gaze = get_gaze_ratio(range(rStart, rEnd), landmarks, gray)
        gaze_ratio = (left_gaze + right_gaze) / 2

        # Interpret gaze
        if gaze_ratio < 0.8:
            gaze = "Right"
        elif gaze_ratio > 1.2:
            gaze = "Left"
        else:
            gaze = "Center"

        # Display gaze direction
        cv2.putText(frame, gaze, (face.left(), face.top()-10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,0,0), 2)

    cv2.imshow("Gaze Detection", frame)
    key = cv2.waitKey(1)
    if key == 27:  # ESC to exit
        break

cap.release()
cv2.destroyAllWindows()

# Cell 6: Results and Cleanup
print("Gaze detection session ended.")
